In [1]:
import utils

In [2]:
LOWER = True
NORM_TENSE = True
REP_NUM = True
REP_YEAR = True
REP_SYM = False

In [3]:
dataset_folder = "data/EN/"
train_data = dataset_folder + "train"
lines = utils.read_file_to_lines(train_data)

In [4]:
sentences = []
pos_tags = []

while len(lines) > 1:
    sentence_break = lines.index("")
    sentence_xy = lines[:sentence_break]
    words = [utils.preprocess_text(token.split(" ")[0],
                                   lower=LOWER,
                                   norm_tense=NORM_TENSE,
                                   replace_number=REP_NUM,
                                   replace_year=REP_YEAR,
                                   replace_symbol=REP_SYM)
             for token in sentence_xy]
    pos = [token.split(" ")[1] for token in sentence_xy]
    sentences.append(words)
    pos_tags.append(pos)
    lines = lines[sentence_break+1:]

In [5]:
o_list = ["a", "the"]
np_list = []

def get_entities(sentence, pos_tag):
    for i in range(len(sentence)):
        try:
            pos = pos_tag[i].split("-")[1]
            if pos == "NP":
                word = sentence[i]
                if word not in o_list and word[0].isalpha():
                    np_list.append(word)
        except Exception as e:
            pass
        
for n in range(len(sentences)):
    sentence = sentences[n]
    pos_tag = pos_tags[n]
    get_entities(sentence, pos_tag)
    
np_list = list(set(np_list))
np_list.sort()
len_np = len(np_list)

In [6]:
import copy
from random import randint, choice

In [ ]:
def rand_bool():
    return choice([True, False])

def augment_line(sentence, pos_tag):
    for i, word in enumerate(sentence):
        try:
            pos = pos_tag[i].split("-")[1]
            if pos == "NP" and sentence[i] not in o_list:
                if rand_bool():
                    r_i = randint(0, len_np)
                    sentence[i] = np_list[r_i]
        except:
            pass
    return sentence

In [15]:
for n in range(len(sentences)):
    sentence = sentences[n]
    print("OG:", " ".join(sentence))
    sentence = copy.deepcopy(sentence)
    pos_tag = pos_tags[n]
    sentence = augment_line(sentence, pos_tag)
    print("AG:", " ".join(sentence))
    break

OG: municipal bond are generally a bit safer than corporate bond in a recession , but not as safe as bond issued by the federal government .
AG: firmness mass-market are generally a bit safer than appropriate bond in a recession , but not as safe as any issued by the heritage government .
